In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE152073"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE152073"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE152073.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE152073.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE152073.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from Brazilian SPAH study"
!Series_summary	"This study is part of previous epidemiologic project, including a population-based survey (Sao Paulo Ageing & Health study (SPAH Study). The data from this study was collected between 2015 to 2016 and involved elderly women (ages ≥65 yeas) living in the Butanta district, Sao Paulo. The purpose of the study was identification of association between transcriptome and the osteo metabolism diseases phenotype, like osteoporosis, vertebral fracture and coronary calcification."
!Series_summary	"Peripheral blood cells suffer alterations in the gene expression pattern in response to perturbations caused by calcium metabolism diseases.   The purpose of this study is to identify possible molecular markers associated with osteoporosis, vertebral fractures and coronary calcification in elderly women from community from Brazilian SPAH study. Vertebral fractures were the most common clinical manife

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset and set availability flags

# 1. Gene expression availability
# Based on the Series Title and Summary, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Determine row keys for trait, age, and gender
# Looking at sample characteristics dictionary

# For trait (osteoporosis), there's no direct mention in the sample characteristics.
# Based on the background information, this study is specifically about osteoporosis in elderly women.
# The entire dataset is about osteoporosis, but we don't have a specific indicator for which 
# subjects have osteoporosis and which don't. Without this differentiation, we can't use the dataset
# for association studies.
trait_row = None

# For age, looking at the sample characteristics, we can see age data is in row 1
age_row = 1

# For gender, we can see from sample characteristics that all samples are female (row 0)
# Since this is a constant feature (all female), we consider it not available
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    # Since we couldn't identify trait data that differentiates subjects, this function won't be used
    return None

def convert_age(value):
    # Extract age value which comes after "age (years): "
    if pd.isna(value):
        return None
    try:
        age_str = value.split(": ")[1]
        return float(age_str)  # Convert to continuous value
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    # This function won't be used as gender is constant, but for completeness:
    if pd.isna(value):
        return None
    try:
        gender = value.split(": ")[1].lower()
        if "female" in gender:
            return 0
        elif "male" in gender:
            return 1
        else:
            return None
    except (IndexError, ValueError):
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip this step


A new JSON file was created at: ../../output/preprocess/Osteoporosis/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Matrix table marker not found in first 100 lines


KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'ID' not found in columns."

Trying alternative approach to read the gene data:


Column names: Index(['GSM4602151', 'GSM4602152', 'GSM4602153', 'GSM4602154', 'GSM4602155'], dtype='object')
First 20 row IDs: Index(['TC01000005.hg.1', 'TC01000006.hg.1', 'TC01000007.hg.1',
       'TC01000008.hg.1', 'TC01000009.hg.1', 'TC01000010.hg.1',
       'TC01000011.hg.1', 'TC01000012.hg.1', 'TC01000013.hg.1',
       'TC01000014.hg.1', 'TC01000015.hg.1', 'TC01000016.hg.1',
       'TC01000017.hg.1', 'TC01000018.hg.1', 'TC01000019.hg.1',
       'TC01000020.hg.1', 'TC01000021.hg.1', 'TC01000022.hg.1',
       'TC01000023.hg.1', 'TC01000024.hg.1'],
      dtype='object', name='ID_REF')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# The identifiers like 'TC01000005.hg.1' appear to be Affymetrix transcript cluster IDs
# These are probe set IDs from the Affymetrix Human Gene ST or similar arrays
# and need to be mapped to standard human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# First properly define gene_data from the previous step
with gzip.open(matrix_file, 'rt') as file:
    # Find the start of the data
    for line in file:
        if '!series_matrix_table_begin' in line:
            break
            
    # Read the data into a DataFrame
    gene_data = pd.read_csv(file, sep='\t', index_col=0)

# 1. Identify the columns in gene_annotation that match our needs
# The 'ID' column in gene_annotation contains the probe identifiers
# The 'gene_assignment' column contains the gene symbol information in a complex format

# 2. Create gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This function will extract human gene symbols from the gene_assignment text
# and handle many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the gene expression data to verify the mapping worked
print("Gene expression data after mapping (first 5 rows, 5 columns):")
print(gene_data.iloc[:5, :5])
print(f"Shape of gene expression data: {gene_data.shape}")


Gene expression data after mapping (first 5 rows, 5 columns):
      GSM4602151  GSM4602152  GSM4602153  GSM4602154  GSM4602155
Gene                                                            
A-     11.755882   11.890018   12.163401   11.986312   11.689942
A-I     1.880696    1.872455    1.861592    1.794649    1.839120
A-II    1.415920    1.302811    1.289576    1.258598    1.275517
A-IV    0.680075    0.685423    0.664525    0.681213    0.689146
A-V     1.070617    1.043819    1.006562    1.083745    1.040385
Shape of gene expression data: (49166, 90)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row is None, trait data is not available
is_trait_available = trait_row is not None
is_gene_available = True

# Create a minimal valid dataframe with the gene data columns
# Using a subset of the gene data to create a valid dataframe structure
minimal_df = pd.DataFrame(index=normalized_gene_data.index[:5], columns=normalized_gene_data.columns[:5])

# Conduct final validation with proper parameters
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=True,  # Without trait data, the dataset is considered biased/unusable
    df=minimal_df,  # Minimal but valid dataframe
    note="Dataset contains gene expression data but lacks per-sample osteoporosis classification."
)

print("Dataset contains gene expression data but lacks sample-level trait information.")
print("Only the normalized gene data was saved. No linked data was created.")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE152073.csv
Dataset contains gene expression data but lacks sample-level trait information.
Only the normalized gene data was saved. No linked data was created.
